## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [4]:
len(nltk_data)

3914

In [5]:
from sklearn.model_selection import train_test_split
import random

In [6]:
#Splitting into train and validation sets in 95:5 ratio
random.seed(1234)
train_set, val_set = train_test_split(nltk_data, train_size=0.95, random_state=42)

In [7]:
len(train_set)

3718

In [8]:
len(val_set)

196

In [9]:
#Creating a list of all (word, tag) tuples
train_tagged_words = [ word_tag_tup for sent in train_set for word_tag_tup in sent ]

print(len(train_tagged_words))
print(train_tagged_words[:5])

95589
[('Bank', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('England', 'NOUN'), ("'s", 'PRT')]


In [10]:
tags_in_data = [tup[1] for tup in train_tagged_words]
tags_in_data[:10]

['NOUN', 'ADP', 'NOUN', 'NOUN', 'PRT', 'NOUN', 'VERB', 'VERB', 'X', 'ADP']

In [11]:
#Printing unique tags in the set
unique_tags = set(tags_in_data)
print(len(unique_tags))
unique_tags

12


{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

#### For emission probabilities

In [12]:
def word_given_tag(word, tag, train_data=train_tagged_words):
    #Getting pairs where tags = 'tag'
    word_tag_list = [tup for tup in train_data if tup[1] == tag]
    #Getting list of word_tag_list pairs for 'word'
    words_given_tag = [tup for tup in word_tag_list if tup[0] == word]
    return len(words_given_tag)/len(word_tag_list)

#### For transmission probabilities in 'tags_in_data'

In [13]:
# 'tags_in_data' is the list of all train tags
total_tag_count = len(tags_in_data)

def tag2_given_tag1(tag2, tag1):
    #Getting list of tags = 'tag1'
    tag1_list = [tag for tag in tags_in_data if tag == tag1]
    #Getting list of tag1_list pairs for 'word'
    count_tag1 = len(tag1_list)
    
    count_tag2_given_tag1 = 0
    for i,tag in enumerate(tags_in_data):
        if i+1<total_tag_count and tags_in_data[i+1] == tag2 and tag==tag1:
            count_tag2_given_tag1+=1
        
    return count_tag2_given_tag1/count_tag1

In [14]:
import numpy as np

#### tags_matrix such that tags_matrix[i][j] has value of P(tj given ti) 

In [15]:
tags_matrix = np.zeros((len(unique_tags),len(unique_tags)), dtype='float32')

for i, t1 in enumerate(unique_tags):
    for j, t2 in enumerate(unique_tags):
        tags_matrix[i][j] = tag2_given_tag1(t2, t1)

In [16]:
tags_matrix

array([[1.72284637e-02, 3.40288915e-02, 8.56072758e-04, 3.21776360e-01,
        8.23970046e-03, 1.31621184e-02, 1.05296947e-01, 3.26377749e-01,
        6.29213452e-02, 1.49812736e-03, 3.94863561e-02, 6.91278726e-02],
       [1.42925426e-01, 7.64818341e-02, 1.03569152e-02, 6.13448061e-02,
        2.03632891e-01, 2.51752716e-02, 1.65710635e-02, 5.51306568e-02,
        2.70873168e-03, 1.85787126e-01, 1.63798600e-01, 5.60866781e-02],
       [5.44345379e-02, 7.97747541e-03, 4.69263265e-04, 3.49131852e-01,
        1.53918341e-01, 5.58423288e-02, 1.16846554e-01, 1.21539183e-01,
        4.22336943e-02, 4.69263270e-03, 3.42562161e-02, 5.86579069e-02],
       [1.76275387e-01, 2.91361269e-02, 4.19356003e-02, 2.64631867e-01,
        1.47977978e-01, 1.68836378e-02, 1.22889541e-02, 1.33099956e-02,
        9.62695573e-03, 4.38318215e-02, 2.39178792e-01, 4.92287474e-03],
       [9.04928222e-02, 2.18005434e-01, 5.58789307e-03, 1.10904150e-01,
        1.69188976e-01, 8.25766400e-02, 6.46488145e-02, 1.33

In [17]:
import pandas as pd

#### Creating a DataFrame of tags as columns and rows with values as transmission probabilities

In [18]:
tags_df = pd.DataFrame(tags_matrix, columns=unique_tags, index=unique_tags)

In [19]:
tags_df

,ADP,X,CONJ,NOUN,VERB,ADV,ADJ,DET,NUM,PRT,.,PRON
ADP,0.017228,0.034029,0.000856,0.321776,0.008240,0.013162,0.105297,0.326378,0.062921,0.001498,0.039486,0.069128
X,0.142925,0.076482,0.010357,0.061345,0.203633,0.025175,0.016571,0.055131,0.002709,0.185787,0.163799,0.056087
CONJ,0.054435,0.007977,0.000469,0.349132,0.153918,0.055842,0.116847,0.121539,0.042234,0.004693,0.034256,0.058658
NOUN,0.176275,0.029136,0.041936,0.264632,0.147978,0.016884,0.012289,0.013310,0.009627,0.043832,0.239179,0.004923
VERB,0.090493,0.218005,0.005588,0.110904,0.169189,0.082577,0.064649,0.133101,0.022817,0.031121,0.035312,0.036244
ADV,0.119601,0.023588,0.006312,0.030897,0.344518,0.081063,0.130233,0.067110,0.030565,0.013621,0.136877,0.015615
ADJ,0.078986,0.021392,0.016949,0.696725,0.012342,0.004608,0.066645,0.005101,0.020405,0.010861,0.065328,0.000658
DET,0.009054,0.045509,0.000483,0.640029,0.038387,0.012313,0.204973,0.005311,0.021970,0.000241,0.017986,0.003742
NUM,0.034790,0.206661,0.013381,0.355338,0.017544,0.002974,0.034196,0.003271,0.184062,0.027951,0.118347,0.001487
PRT,0.021576,0.013403,0.002288,0.242563,0.402746,0.010134,0.086303,0.100360,0.058516,0.001635,0.041517,0.018960


### Build the vanilla Viterbi based POS tagger

In [20]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            emission_p = word_given_tag(word, tag)
            
            # compute emission and state probabilities
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))


In [21]:
# List of (word, tags) in validation set
val_run_base = [tup for sent in val_set for tup in sent]
val_run_base[:20]

[('For', 'ADP'),
 ('the', 'DET'),
 ('Agency', 'NOUN'),
 ('for', 'ADP'),
 ('International', 'NOUN'),
 ('Development', 'NOUN'),
 (',', '.'),
 ('appropriators', 'NOUN'),
 ('approved', 'VERB'),
 ('$', '.'),
 ('200', 'NUM'),
 ('million', 'NUM'),
 ('*U*', 'X'),
 ('in', 'ADP'),
 ('secondary', 'ADJ'),
 ('loan', 'NOUN'),
 ('guarantees', 'NOUN'),
 ('under', 'ADP'),
 ('an', 'DET'),
 ('expanded', 'VERB')]

In [22]:
#Creating a list of words from the validation set

val_tagged_words = [tup[0] for sent in val_set for tup in sent]
val_tagged_words[:10]

['For',
 'the',
 'Agency',
 'for',
 'International',
 'Development',
 ',',
 'appropriators',
 'approved',
 '$']

In [23]:
# Tagging words in the validation set using Vanilla Viterbi
tagged_seq = Viterbi(val_tagged_words)

In [24]:
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j] 

### Accuracy of Vanilla Viterbi

In [25]:
len(check)/len(tagged_seq)

0.9146844898761549

#### Creating a list of previous (word, tag), predicted and actual (word,tag) of the validation set

In [26]:
incorrect_tagged_cases = [{ 'prev': val_run_base[i-1], 'predicted': j[0], 'actual': j[1]} for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases[:10]

[{'prev': ('expanded', 'VERB'),
  'predicted': ('trade', 'VERB'),
  'actual': ('trade', 'NOUN')},
 {'prev': ('the', 'DET'),
  'predicted': ('Overseas', 'ADP'),
  'actual': ('Overseas', 'NOUN')},
 {'prev': ('Overseas', 'NOUN'),
  'predicted': ('Private', 'ADJ'),
  'actual': ('Private', 'NOUN')},
 {'prev': ('settled', 'VERB'),
  'predicted': ('pre-1917', 'ADP'),
  'actual': ('pre-1917', 'ADJ')},
 {'prev': ('``', '.'),
  'predicted': ('Unemployment', 'ADP'),
  'actual': ('Unemployment', 'NOUN')},
 {'prev': ('the', 'DET'),
  'predicted': ('purchasing', 'NOUN'),
  'actual': ('purchasing', 'VERB')},
 {'prev': ('weekly', 'ADJ'),
  'predicted': ('paycheck', 'ADP'),
  'actual': ('paycheck', 'NOUN')},
 {'prev': ('paycheck', 'NOUN'),
  'predicted': ('reasonably', 'ADP'),
  'actual': ('reasonably', 'ADV')},
 {'prev': (',', '.'),
  'predicted': ('though', 'ADP'),
  'actual': ('though', 'ADV')},
 {'prev': ('such', 'ADJ'),
  'predicted': ('close', 'NOUN'),
  'actual': ('close', 'ADJ')},
 {'prev': ('a

### Analysing the incorrectly tagged words

#### Word chain with '-' in them must be tagged as 'ADJ'
#### Eg: 'American-style', 'cross-border', 'pre-existing'

#### Words ending with 'ed' and 'ing' :VERB
#### Eg: 'waived', 'shopped', 'alleged', 'indulging', 'apologizing'

#### Words with digits in them decimal or non-decimal must be tagged as 'NUM'
#### Eg: '1955', '133.7', '94'

#### Words with '*-' in them must be tagged as 'X'
#### Eg: '\*T\*-133 ', '\*T\*-253 ', '\*-130 '

In [27]:
incorrectly_assigned_tags = [obj['predicted'][1] for obj in incorrect_tagged_cases]
incorrectly_assigned_tags[:10]

['VERB',
 'ADP',
 'ADJ',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'ADP',
 'ADP',
 'NOUN',
 'ADV',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADV',
 'ADP',
 'NOUN',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADJ',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'CONJ',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADJ',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADV',
 'ADP',
 'ADJ',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'VERB',
 'PRT',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'PRT',
 'ADP',
 'ADP',
 'ADJ',
 'ADP',
 'NOUN',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'VERB',
 'VERB',
 'VERB',
 'DET',
 'NOUN',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'NOUN',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'NOUN',
 'NOUN',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADJ',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'NOUN',
 'ADP',
 'PRT',
 'ADP',
 'ADP',
 'ADP',
 'ADP',
 'NOUN',
 'AD

In [28]:
#Occurences of each incorrectly assigned tags in Vanilla Viterbi
freq_dist = { tag: incorrectly_assigned_tags.count(tag) for tag in set(incorrectly_assigned_tags)}
freq_dist

{'ADP': 326,
 'CONJ': 1,
 'NOUN': 35,
 'VERB': 25,
 'ADV': 16,
 'ADJ': 20,
 'DET': 3,
 'NUM': 3,
 'PRT': 5}

#### From the distribution, it is observed that the first tag 'ADP' in the set is assigned to an unknown word by default

In [29]:
freq_dist_train = { tag: tags_in_data.count(tag) for tag in unique_tags}
freq_dist_train

{'ADP': 9345,
 'X': 6276,
 'CONJ': 2131,
 'NOUN': 27423,
 'VERB': 12885,
 'ADV': 3010,
 'ADJ': 6077,
 'DET': 8284,
 'NUM': 3363,
 'PRT': 3059,
 '.': 11118,
 'PRON': 2618}

In [30]:
100*(freq_dist_train['NOUN']/len(tags_in_data))

28.6884474154976

#### 'NOUN' tag occurs the most at 28.68% in the train data 

### Solve the problem of unknown words

### Approach 1

#### Solving the problem of unknown words by tagging them as 'NOUN' instead of the fist tag in the set

In [31]:
# Viterbi Heuristic by tagging words as 'NOUN' for unknown words
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            emission_p = word_given_tag(word, tag)
            
            # compute emission and state probabilities
            state_probability = emission_p * transition_p
            p.append(state_probability)
            
        pmax = max(p)
        
        if pmax == 0.0:
            state_max = 'NOUN'            # For unknown words (state_probability=0), tagging them as 'NOUN' 
        else:
            state_max = T[p.index(pmax)]  # getting state for which probability is maximum
        state.append(state_max)
    return list(zip(words, state))

In [32]:
tagged_seq_iter1 = Viterbi_1(val_tagged_words)

In [33]:
check1 = [i for i, j in zip(tagged_seq_iter1, val_run_base) if i == j] 

### Accuracy of modified Viterbi


In [34]:
len(check1)/len(tagged_seq_iter1)

0.9396500884607824

### Accuracy increased from 91.46% to 93.96% 
#### Tagging unknown words with frequently occuring tag in the train helps in tagging correctly to an extent

### Predicted by Vanilla Viterbi:
#### 'predicted': ('protocols', 'ADP')
#### 'actual': ('protocols', 'NOUN')


#### 'predicted': ('Overseas', 'ADP')
#### 'actual': ('Overseas', 'NOUN')


#### 'predicted': ('paycheck', 'ADP')
#### 'actual': ('paycheck', 'NOUN')

### Predicted by Modification 1

In [36]:
def find_tag_given_word_from_output(word):
    return [tup for tup in tagged_seq_iter1 if tup[0]==word]

In [68]:
find_tag_given_word_from_output('protocols')

[('protocols', 'NOUN')]

In [66]:
find_tag_given_word_from_output('Overseas')

[('Overseas', 'NOUN')]

In [67]:
find_tag_given_word_from_output('paycheck')

[('paycheck', 'NOUN')]

#### So instead of getting tagged as the first occuring tag in the set,
#### assigning the unkown words to most occuring tag, i.e 'NOUN' has helped fixing the tag assignments

### Approach 2

#### Trying out regex tagger as a backoff to the unknown words (whose state_probability is 0)
#### Using the pattern found in incorrect_tagged_cases (from Vanilla Viterbi)

In [40]:
patterns=[
    (r'\d+.?\d*$','NUM'),      # digits with or without decimals should be tagged as 'NUM'
    (r'.*\*-','X'),            # words with '*-' in them tagged as 'X'
    (r'.*-.*','ADJ'),          # words with '-' in them, like '' are tagged 'ADJ'
    (r'.*ing$|.*ed$','VERB')   # words ending with 'ing' and 'ed' are 'VERB'
]

In [41]:
# RegexpTagger initialised with patterns
regexp_tagger = nltk.RegexpTagger(patterns)

# Viterbi Heuristic that tags words:
#  1. Using Viterbi
#  2. If state_probability is 0, then tags using RegexpTagger
#  3. If even RegexpTagger can't find a tag to the word, tag it as 'NOUN' 
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            emission_p = word_given_tag(word, tag)
            
            # compute emission and state probabilities
            state_probability = emission_p * transition_p
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = ''
        if pmax != 0.0:
            state_max = T[p.index(pmax)]
        else:
            state_max = regexp_tagger.tag([word])[0][1]
        if not state_max:
            state_max = 'NOUN'
        state.append(state_max)
    return list(zip(words, state))


In [42]:
tagged_seq_iter2 = Viterbi_2(val_tagged_words)

In [43]:
check2 = [i for i, j in zip(tagged_seq_iter2, val_run_base) if i == j] 

### Accuracy for the modified Viterbi

In [44]:
len(check2)/len(tagged_seq_iter2)

0.961666994299194

### Accuracy increased from 91.46% to 96.16%
#### Tagging unknown words using RegexpTagger definitely helped
#### Tagging unknown words with frequently occuring tag in the train set helps in tagging correctly to an extent

### Predicted by Vanilla Viterbi:

### NOUN
#### 'predicted': ('protocols', 'ADP') => 'actual': ('protocols', 'NOUN')

#### 'predicted': ('Overseas', 'ADP') => 'actual': ('Overseas', 'NOUN')

### ADJ
#### 'predicted': ('American-style', 'ADP') =>  'actual': ('American-style', 'ADJ')

#### 'predicted': ('cross-border', 'ADP') =>  'actual': ('cross-border', 'ADJ')

### VERB
#### 'predicted': ('waived', 'ADP') =>  'actual': ('waived', 'VERB')

#### 'predicted': ('shopped', 'ADP') =>  'actual': ('shopped', 'VERB')
       
#### 'predicted': ('indulging', 'ADP') =>  'actual': ('purchasing', 'VERB')

#### 'predicted': ('apologizing', 'ADP') =>  'actual': ('apologizing', 'VERB')

### NUM
#### 'predicted': ('1955', 'ADP') =>  'actual': ('1955', 'NUM')

#### 'predicted': ('133.7', 'ADP') =>  'actual': ('133.7', 'NUM')

### X
#### 'predicted': ('\*T\*-133', 'ADP') =>  'actual': ('\*T\*-133', 'X')

#### 'predicted': ('\*-130', 'ADP') =>  'actual': ('\*-130', 'X')

### Predicted by Modification 2

In [46]:
def find_tag_given_word_from_output(word):
    return [tup for tup in tagged_seq_iter2 if tup[0]==word]

In [ ]:
# 'NOUN'

In [69]:
find_tag_given_word_from_output('protocols')

[('protocols', 'NOUN')]

In [70]:
find_tag_given_word_from_output('Overseas')

[('Overseas', 'NOUN')]

In [ ]:
# 'ADJ'

In [71]:
find_tag_given_word_from_output('American-style')

[('American-style', 'ADJ')]

In [72]:
find_tag_given_word_from_output('cross-border')

[('cross-border', 'ADJ')]

In [ ]:
# 'VERB'

In [73]:
find_tag_given_word_from_output('waived')

[('waived', 'VERB')]

In [74]:
find_tag_given_word_from_output('shopped')

[('shopped', 'VERB')]

In [76]:
find_tag_given_word_from_output('indulging')

[('indulging', 'VERB')]

In [78]:
find_tag_given_word_from_output('apologizing')

[('apologizing', 'VERB')]

In [ ]:
# 'NUM'

In [80]:
find_tag_given_word_from_output('1955')

[('1955', 'NUM')]

In [81]:
find_tag_given_word_from_output('133.7')

[('133.7', 'NUM')]

In [ ]:
# 'X'

In [82]:
find_tag_given_word_from_output('*T*-133')

[('*T*-133', 'X')]

In [83]:
find_tag_given_word_from_output('*-130')

[('*-130', 'X')]

### Test sentences

In [57]:
f= open('Test_sentences.txt')
test_file_content=f.read()

In [58]:
test_file_content

"Android is a mobile operating system developed by Google.\nAndroid has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\nGoogle and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\nTwitter is an online news and social networking service on which users post and interact with messages known as tweets.\nBefore entering politics, Donald Trump was a domineering businessman and a television personality.\nThe 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\nThis is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\nShow me the cheapest round trips from Dallas to Atlanta\nI would like to see flights from Denver to Philadelphia.\nShow me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\nNASA invited social media users to experience the launch of ICESAT-2 Satell

In [59]:
#tokenizing the text
from nltk.tokenize import word_tokenize

In [60]:
test_words = word_tokenize(test_file_content)
test_words[:10]

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.']

#### Assigning tags to test sample using Vanilla Viterbi and the two modifications

In [61]:
#Vanilla Viterbi
tagged_test = Viterbi(test_words)

#Modification 1
tagged_test_1 = Viterbi_1(test_words)

#Modification 2
tagged_test_2 = Viterbi_2(test_words)


In [62]:
tagged_test[:15]

[('Android', 'ADP'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'ADP'),
 ('.', '.'),
 ('Android', 'ADP'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'ADP'),
 ('worldwide', 'ADP'),
 ('on', 'ADP'),
 ('smartphones', 'ADP'),
 ('since', 'ADP'),
 ('2011', 'ADP'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'ADP'),
 ('.', '.'),
 ('Google', 'ADP'),
 ('and', 'CONJ'),
 ('Twitter', 'ADP'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'ADP'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'ADP'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'ADP'),
 ("'s", 'PRT'),
 ('firehose', 'ADP'),
 ('.', '.'),
 ('Twitter', 'ADP'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADP'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOU

#### 'Android', 'Google' and 'Twitter' are tagged as 'ADP' should be a 'NOUN'

#### '2013' and '2018' are tagged as 'ADP' should be a 'NUM'

#### 'contested' is tagged as 'ADP' shuold be 'VERB'

In [63]:
# Predictions by Modification 1
tagged_test_1[:15]

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NOUN'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NOUN'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NOUN'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 

#### 'Android', 'Google' and 'Twitter' are tagged as 'NOUN'

#### But '2013' and '2018' are tagged as 'NOUN' should be a 'NUM'

#### And 'contested' is tagged as 'NOUN' shuold be 'VERB'

In [64]:
# Predictions by Modification 2
tagged_test_2[:15]

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

#### 'Android', 'Google' and 'Twitter' are tagged correctly as 'NOUN'

#### '2013' and '2018' are tagged correctly as 'NUM'

#### 'contested' is correctly tagged as 'VERB'